In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import imblearn
from imblearn.over_sampling import SMOTE

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier, Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report

In [51]:
data = pd.read_parquet("../data/df_cleaned.parquet")
data

,transaction_date,amount,memo,new_category,Cleaned Text
8,2018-03-06,15.25,POS CASINO BAR @ SPOTL - MEMO=PURCHASE 03/02 C...,Entertainment,pos casino bar spotl memopurchase 0302 coache...
39,2018-09-29,16.04,BEST BUY GRAND REGENCY BRANDON F,General Merchandise,best buy grand regency brandon f
45,2018-10-17,10.36,CORNER STORE ARLINGTON TX 10/17 Purchase $5.3...,Automotive,corner store arlington tx 1017 purchase $536 ...
52,2017-07-11,4.63,SPEEDWAY IN BEDFORD IN 07/10 DEBIT_CARD,Automotive,speedway bedford 0710 debitcard
55,2018-09-10,223.00,PAYMENT FOR AMZ STORECARD WEB ID: ACH_DEBIT,General Merchandise,payment amz storecard web id achdebit
...,...,...,...,...,...
1999981,2021-02-19,24.24,DBT/WDR CASEYS GEN STORE FENNIMORE WI,Automotive,dbtwdr caseys gen store fennimore wi
1999982,2021-01-19,120.60,POS Debit - DDA KOHLS CLIVE IA #,General Merchandise,pos debit dda kohls clive ia
1999983,2022-02-25,6.47,STARBUCKS STORE MAPLE VALLEY WA 02/24,Food and Beverages,starbucks store maple valley wa 0224
1999991,2021-01-11,44.98,POS Debit - Visa Check Card - APPLE.COM/BILL ...,General Merchandise,pos debit visa check card applecombill ca


In [34]:
data['new_category'].value_counts(normalize=True)

Food and Beverages     0.326565
General Merchandise    0.280126
Automotive             0.128234
Groceries              0.128081
Entertainment          0.063479
Travel                 0.051383
Healthcare/Medical     0.016916
Pets/Pet Care          0.005217
Name: new_category, dtype: float64

# Part 1: TF-IDF Model

##### Koosha's TF-IDF model + Nathan's fine-tuning

In [35]:
# define the feature columns
feature_columns = "Cleaned Text"

# define the target column
target_column = "new_category"

# Create a LabelEncoder object
le = LabelEncoder()

# Fit and transform the target column
y = le.fit_transform(data[target_column])

# create the feature matrix and target vector
X = data[feature_columns]

# split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [36]:
# le.classes_
for i in range(8):
    print(i, le.inverse_transform([i]))

0 ['Automotive']
1 ['Entertainment']
2 ['Food and Beverages']
3 ['General Merchandise']
4 ['Groceries']
5 ['Healthcare/Medical']
6 ['Pets/Pet Care']
7 ['Travel']


In [37]:
# create the TfidfVectorizer
vectorizer = TfidfVectorizer()

# create the pipeline
pipeline = Pipeline([
    ("vectorizer", vectorizer),
    ("classifier", LogisticRegression())
])

# fit the model on the training data
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', LogisticRegression())])

In [38]:
# test the model on the test data
test_data = "autozone"
predicted_category = pipeline.predict([test_data])

print("Predicted category:", le.inverse_transform(predicted_category))

Predicted category: ['Automotive']


In [39]:
# evaluate the model on the test data
score = pipeline.score(X_test, y_test)
print("Test score:", score)

Test score: 0.8560599582198224


### Parameter Tuning

In [40]:
def trainTestTFIDF(gram='word', ngram_range=(1,1)):
        # create the TfidfVectorizer
    vectorizer = TfidfVectorizer(analyzer=gram, ngram_range=ngram_range)

    # create the pipeline
    pipeline = Pipeline([
        ("vectorizer", vectorizer),
        ("classifier", LogisticRegression())
    ])

    # fit the model on the training data
    pipeline.fit(X_train, y_train)
    
    # evaluate the model on the test data
    score = pipeline.score(X_test, y_test)
    print("Test score:", score)
    return pipeline

In [41]:
# trainTestTFIDF('word')
# trainTestTFIDF('word', ngram_range=(2,2)) # bigrams
# trainTestTFIDF('word', ngram_range=(3,3)) # trigrams
# trainTestTFIDF('char')
# trainTestTFIDF('char', ngram_range=(2,2))
# trainTestTFIDF('char', ngram_range=(3,3))
# trainTestTFIDF('char', ngram_range=(4,4))

best_tfidf = trainTestTFIDF('char', ngram_range=(5,5)) # BEST
#  '123 4567' includes: '123 4', '23 45', '3 456', ' 4567'

# trainTestTFIDF('char', ngram_range=(6,6))
# trainTestTFIDF('char', ngram_range=(7,7))
# trainTestTFIDF('char', ngram_range=(8,8))

tfidf_proba = best_tfidf.predict_proba(X_test)

Test score: 0.8746594671194888


# Part 2: Non-Text Model
##### Kyle's Non-Text Model

In [52]:
data['dt'] = pd.to_datetime(data['transaction_date'])
data = data[["amount", "dt", "new_category"]]
data

,amount,dt,new_category
8,15.25,2018-03-06,Entertainment
39,16.04,2018-09-29,General Merchandise
45,10.36,2018-10-17,Automotive
52,4.63,2017-07-11,Automotive
55,223.00,2018-09-10,General Merchandise
...,...,...,...
1999981,24.24,2021-02-19,Automotive
1999982,120.60,2021-01-19,General Merchandise
1999983,6.47,2022-02-25,Food and Beverages
1999991,44.98,2021-01-11,General Merchandise


In [53]:
# feature engineering using dt column: adding year, month, day, etc.
data['year'] = data['dt'].dt.year # ranges 2010 - 2022
data['month'] = data['dt'].dt.month # ranges 1 - 12
data['is_holiday'] = data['month'].apply(lambda x: 1 if x == 11 or x == 12 else 0) # 0 or 1
data['day'] = data['dt'].dt.dayofweek # to be discarded
data['is_weekend'] = data['day'].apply(lambda x: 1 if x == 5 or x == 6 else 0) # 0 or 1

# feature engineering using amount: encoding whole numbers
data['is_whole_number'] = data['amount'].apply(lambda x: 1 if str(x).split(".")[1] == "0" else 0)
data

# X y split
X = data[["amount", "is_whole_number", "year", "month", "day", "is_holiday", "is_weekend"]]
y = data[["new_category"]]

# standard scale: amount
scaler = StandardScaler()
scaler_df = pd.DataFrame(scaler.fit_transform(X[['amount']]), index = X.index)

# one hot encode: year, month, day
ohe = OneHotEncoder(handle_unknown='ignore')
encoder_df = pd.DataFrame(ohe.fit_transform(X[['year']]).toarray(), columns = pd.Series(X['year'].unique()).sort_values().values, index = X.index)

ohe2 = OneHotEncoder(handle_unknown='ignore')
encoder_df2 = pd.DataFrame(ohe2.fit_transform(X[['month']]).toarray(), columns = pd.Series(X['month'].unique()).sort_values().values, index = X.index)

# monday = 0, sunday = 6
def convert_date_to_day(date):
    if date == 0:
        return "monday"
    if date == 1:
        return "tuesday"
    if date == 2:
        return "wednesday"
    if date == 3:
        return "thursday"
    if date == 4:
        return "friday"
    if date == 5:
        return "saturday"
    if date == 6:
        return "sunday"

X['day_word'] = X['day'].apply(convert_date_to_day)
ohe3 = OneHotEncoder(handle_unknown='ignore')
encoder_df3 = pd.DataFrame(ohe3.fit_transform(X[['day_word']]).toarray(), columns = pd.Series(X['day_word'].unique()).sort_values().values, index = X.index)

# combine features and remove duplicate features
X = pd.concat([X, scaler_df, encoder_df, encoder_df2, encoder_df3], axis=1)
X = X[[0, 'is_whole_number', 'is_holiday', 'is_weekend', 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, "monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]]
X.rename({0: "amount (standardized)"}, axis=1, inplace=True)

In [54]:
# train test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [57]:
# train model
lr = LogisticRegression(multi_class = "multinomial").fit(X_train, y_train)
y_train = le.transform(y_train)
rr = Ridge(alpha = 1).fit(X_train, y_train)
# sgd = SGDClassifier(loss="log").fit(X_train, y_train)

# predict and store predictions
lr_preds = le.transform(lr.predict(X_test))
# sgd_preds = sgd.predict(X_test)

y_test = np.array(y_test['new_category'])
y_test_encoded = le.transform(y_test)
y_test_encoded

array([0, 7, 2, ..., 2, 0, 7])

In [61]:
rr_preds = rr.predict(X_test)
pd.Series(rr_preds).sort_values()

101417    1.754446
152365    1.756610
109641    2.285188
29174     2.287831
144643    2.287851
            ...   
111460    2.643557
61629     2.643758
128326    2.645205
146273    2.810262
40496     4.546615
Length: 163714, dtype: float64

In [ ]:
lr_accuracy = (lr_preds == y_test_encoded).mean()
# sgd_accuracy = (sgd_preds == y_test_encoded).mean()
print("Logistic Regression accuracy: {}, \nSGDClassifier accuracy: {}".format(lr_accuracy, sgd_accuracy))


# Part 3: Composite Model
##### Kyle using Logistic Regression to combine the probabilities from each class from both models

In [62]:
tfidf_proba, lr_proba, rr_preds

NameError: name 'lr_proba' is not defined

In [ ]:
# lr_proba = lr.predict_proba(X_test)
# # final_model = LogisticRegression().fit()
# for weight in range(0,20): # learn weight
#     weighted_average = nontext_proba + weight*tfidf_proba
#     final_df = pd.DataFrame(weighted_average)
# # todo: do a sanity check on this model. look at a specific transaction and trace it
#     final_model = LogisticRegression().fit(final_df, y_test)
    
#     preds = final_model.predict(final_df)
#     print(weight, np.mean(preds == y_test))

In [63]:
# COMPOSITE MODEL DOES NOT WORK
# next week:
# -kyle: think about trying ridge regression
# -try to combine the X_train non-text and text data rather than combining the predictions
# -both text and non-text were already trained on LR so shouldn't be too difficult
# -might have to take it out of the pipeline
# -start working on bert and gpt